### All necessary imports

In [18]:
#import urllib.request
#from urllib.request import urlopen
import urllib
import os
import string
import glob
import shutil
import csv
import pandas as pd
import numpy as np
from zipfile import ZipFile
from tempfile import NamedTemporaryFile
from shutil import unpack_archive
from io import BytesIO
import logging
from boto.s3.key import Key
import ntpath
import zipfile
import sys
from os.path import basename

In [19]:
# Here I created handler, formatter, loglevel etc..
ErrorLog = 'errorLoggingP2.log'
log = logging.basicConfig(filename=ErrorLog,level=logging.DEBUG, format='%(asctime)s- %(levelname)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', filemode = 'w')#ch1 = logging.FileHandler('ErrorLog') #output the logs to a file

In [20]:
directory = os.path.dirname(os.getcwd()) 
cur_path =  directory + '/Part_2'
newpath = '/Users/sonalichaudhari/Desktop/FALL2017/ADS/Assignment_1/Part_2/LogCSV'

In [21]:
#
year='2004'
AWS_ACCESS_KEY_ID =''
AWS_SECRET_ACCESS_KEY =''

for i in range(1,len(sys.argv)):
    val=sys.argv[i]
    if val.startswith('year='):
        pos=val.index("=")
        year=val[pos+1:len(val)]
        continue
    elif val.startswith('accessKey='):
        pos=val.index("=")
        AWS_ACCESS_KEY_ID=val[pos+1:len(val)]
        continue
    elif val.startswith('secretKey='):
        pos=val.index("=")
        AWS_SECRET_ACCESS_KEY=val[pos+1:len(val)]
        continue

In [22]:
#
if not AWS_ACCESS_KEY_ID or not AWS_SECRET_ACCESS_KEY:
    logging.warning('no access or secret key provided')
    print('No access or secret key provided')
    #exit()
try:
    conn = boto.connect_s3(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)
    logging.info("Successfully connected to Amazon S3")

except:
    logging.info("Invalid amazon keys")
    print("Invalid amazon keys")
    #exit()

No access or secret key provided
Invalid amazon keys


### Downloading the files in zip

In [23]:
# Unzip and downloading the files in the folder

def openUnzip(url):
    try:
        if not os.path.exists(newpath):
            os.makedirs(newpath)
            with urlopen(url) as zipresp:
                with ZipFile(BytesIO(zipresp.read())) as zfile:
                    zfile.extractall(newpath)
        else:
            with urlopen(url) as zipresp:
                with ZipFile(BytesIO(zipresp.read())) as zfile:
                    zfile.extractall(newpath)
    except:
        logging.warning("url:" + url + " was not correct. Check the inputted year")         

In [24]:
#year = 
if(year is None):
     logging.warning("year was left blank. Assigning the year 2005 automatically")
else:
     logging.info("year was set properly")
zipurl = 'http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/'
generalUrl = "http://www.sec.gov/dera/data/PublicEDGAR-log-file-data/"
months = ['01',"02","03","04","05","06","07","08","09","10","11","12"]
for m in range(len(months)):
    if(str(months[m]) is "01" or str(months[m]) is "02" or str(months[m]) is "03"):
        urlAppended = zipurl + str(year) +"/Qtr1/"+"log"+str(year)+str(months[m])+"01.zip"
        #print(urlAppended)
        openUnzip(urlAppended)
    if(str(months[m]) is "04" or str(months[m]) is "05" or str(months[m]) is "06"):
        urlAppended = zipurl + str(year) +"/Qtr2/"+"log"+str(year)+str(months[m])+"01.zip"
        #print(urlAppended)
        openUnzip(urlAppended)
    if(str(months[m]) is "07" or str(months[m]) is "08" or str(months[m]) is "09"):
        urlAppended = zipurl + str(year) +"/Qtr3/"+"log"+str(year)+str(months[m])+"01.zip"
        #print(urlAppended)
        openUnzip(urlAppended)
    if(str(months[m]) is "10" or str(months[m]) is "11" or str(months[m]) is "12"):
        urlAppended = zipurl + str(year) +"/Qtr4/"+"log"+str(year)+str(months[m])+"01.zip"
        #print(urlAppended)
        openUnzip(urlAppended)

### Reading the CSV in a dataframe

### Reading all CSVs and storing it in a dictionary

In [25]:
all_log_files = glob.glob(newpath+'/*.csv')
dict_csv = {}
for file_ in all_log_files:
    f_name = os.path.basename(file_)
    dict_csv[f_name] = pd.read_csv(file_,index_col=None, header=0)

logging.info("Each csv has been read into a dataframe ")

In [ ]:
dict_csv.keys()

##  Data Wrangling

In [26]:
for key,value in dict_csv.items():
    df = dict_csv[key]
    
    #Dropping Cik,ip and accesion if NAN
    df.dropna(subset=['cik'])
    df.dropna(subset=['accession'])
    df.dropna(subset=['ip'])
    
    # df.isnull().sum()

    #Extension : if only extension is there, appending accession before the extension
    df['extention'] = np.where(str(df['extention']).find('.') ==0,df['extention'],df['accession']+df['extention'])
    logging.info("replaced all extensions with no extension name in front of . with accession number")
    df['extention'] = df['extention'].replace(np.nan, 'Unavailable', regex=True)
    logging.info("replaced all extensions with NAN by 'Unavailable'")

    #Zone : if NAN fill it with the max used zone value
    df['zone'] = df['zone'].fillna(df['ip'].value_counts().idxmax())
    logging.info("replaced all zones with NAN with mazimum count of ip address")
    
    #IDX : if NAN fill it with the max used idx value
    df['idx'] = df['idx'].fillna(df['idx'].value_counts().idxmax())
    logging.info("replaced all idx with NAN with value that max count")
    
    #Norefer : if NAN fill it with the max used norefer value
    df['norefer'] = df['norefer'].fillna(df['norefer'].value_counts().idxmax())
    logging.info("replaced all norefer with NAN with value that max count")

    #Noagent : if NAN fill it with the max used noagent value
    df['noagent'] = df['noagent'].fillna(df['noagent'].value_counts().idxmax())
    logging.info("replaced all noagent with NAN with value that max count")

    #Crawler : if NAN fill it with the max used crawler value
    df['crawler'] = df['crawler'].fillna(df['crawler'].value_counts().idxmax())
    logging.info("replaced all crawler with NAN with value that max count")

    #Replace size column which has NAN with the mean file size
    df['size'] = df['size'].fillna(df['size'].mean(axis=0))

    # df['date'].notnull().value
    # #sub_df.iloc[0]['A']

    #Browser : if NAN fill it with the max used browswe value
    df['browser'] = df['browser'].fillna(df['browser'].value_counts().idxmax())


### Summary

In [ ]:
# Mean File size
#df['size_mean'] = df['size'].mean(axis=0)
# Unique IPs in the given month
#df['ip_unique'] = df['ip'].nunique()
#df.describe()
#Unique CIK
#df['cik'].nunique()
#Unique Accession Number
#df['accession'].nunique()

### Concating all the dataframe 

In [27]:
mdf = pd.concat(dict_csv)
mdf.to_csv(newpath+'Combined_CSV.csv')

In [28]:
a = mdf.head(20)
a.head()

ip        date      time   zone        cik  \
log20040101.csv 0     24.70.95.bjg  2004-01-01  00:00:00  500.0   771252.0   
                1   64.165.202.fca  2004-01-01  00:00:00  500.0   849778.0   
                2  207.168.174.jdd  2004-01-01  00:00:01  500.0  1244190.0   
                3   64.165.202.fca  2004-01-01  00:00:02  500.0   849778.0   
                4   64.165.202.fca  2004-01-01  00:00:03  500.0   849778.0   

                              accession                             extention  \
log20040101.csv 0  0001047469-03-042434  0001047469-03-042434a2125426zs-3.htm   
                1  0000927016-03-001282       0000927016-03-001282dex1014.txt   
                2  0001244190-03-000001         0001244190-03-000001edgar.xml   
                3  0000927016-03-001282       0000927016-03-001282dex1015.txt   
                4  0000927016-03-001282        0000927016-03-001282dex211.htm   

                    code      size  idx  norefer  noagent  find  crawler  \
log20040101.csv 0  200.0  123558.0  0.0      0.0      0.0  10.0      0.0   
                1  200.0   38688.0  0.0      1.0      0.0   0.0      0.0   
                2  200.0    5683.0  0.0      1.0      1.0   0.0      0.0   
                3  200.0   17038.0  0.0      1.0      0.0   0.0      0.0   
                4  200.0    9025.0  0.0      1.0      0.0   0.0      0.0   

                  browser  
log20040101.csv 0     win  
                1     win  
                2     win  
                3     win  
                4     win

In [ ]:
#kk = mdf['cik'].value_counts(sort=True, ascending=False,)
cik_ = a.groupby(['cik']).size().nlargest(10)
#cik_

In [ ]:
mdf['ip'].nunique()

In [ ]:
a['Quarter']= a['date'][0][5:7]

In [ ]:
for r,i in a.iterrows():
    month = a['date'][r][5:7]
    x = int(month)
    if x < 4:
        a['Quarter'] = 'QTR1'
    elif (x > 3) and (x < 7):
        a["Quarter"] = 'QTR2'
    elif x >7 and x < 10:
        a['Quarter'] = 'QTR3'
    else:
        a['Quarter'] = 'QTR4'

In [ ]:
a